In [ ]:
# NARX from library PyNeurGen
# code for NARXRecurrent provided by: Copyright (C) 2012  Don Smiley  ds@sidorof.com

In [ ]:
from pyneurgen.neuralnet import NeuralNet
from pyneurgen.nodes import CopyNode, Connection
from pyneurgen.nodes import NODE_OUTPUT, NODE_HIDDEN, NODE_INPUT
from pyneurgen.nodes import NODE_BIAS, ACTIVATION_LINEAR

In [ ]:
class NARXRecurrent(RecurrentConfig):
    """
    This class implements a process for converting a standard neural network
    into a NARX (Non-Linear AutoRegressive with eXogenous inputs) recurrent
    network.

    It also contains some modifications suggested by Narendra and Parthasathy
    (1990).
    
    """

    def __init__(self, output_order, incoming_weight_from_output,
                      input_order, incoming_weight_from_input):
        """
        This function takes:
            the output order, or number of copy levels of
                output values,
            the weight to apply to the incoming values from output nodes,
            the input order, or number of copy levels of input values,
            the weight to apply to the incoming values from input nodes

        """
        RecurrentConfig.__init__(self)
        self.existing_weight = 0.0
        self._node_type = None

        self.output_values = [output_order, incoming_weight_from_output]
        self.input_values = [input_order, incoming_weight_from_input]

    def get_source_nodes(self, neural_net):
        """
        This function returns either the output nodes or input nodes depending
        upon self._node_type.

        """

        if self._node_type == NODE_OUTPUT:
            return neural_net.layers[-1].get_nodes(self._node_type)
        elif self._node_type == NODE_INPUT:
            return neural_net.layers[0].get_nodes(self._node_type)

    def apply_config(self, neural_net):
        """
        This function first applies any parameters related to the output nodes
        and then any with the input nodes.

        """

        if self.output_values[0] > 0:
            self._node_type = NODE_OUTPUT
            self.copy_levels = self.output_values[0]
            self.incoming_weight = self.output_values[1]

            self._apply_config(neural_net)

        if self.input_values[0] > 0:
            self._node_type = NODE_INPUT
            self.copy_levels = self.input_values[0]
            self.incoming_weight = self.input_values[1]

            self._apply_config(neural_net)

In [ ]:
import random
import math
import numpy as np
from pandas import read_csv
import matplotlib 
import matplotlib.pyplot as plt
from pyneurgen.recurrent import NARXRecurrent
from pyneurgen.nodes import BiasNode, Connection

In [ ]:
# first run load data
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
trainY=numpy.reshape(trainY,(trainY.shape[0],1))
testY=numpy.reshape(testY,(testY.shape[0],1))

In [ ]:
net = NeuralNet()

# Set the NARX architecture
input_nodes = 2
hidden_nodes = 10
output_nodes = 2

output_order = 3
incoming_weight_from_output = .4
input_order = 2
incoming_weight_from_input = .6

# constrain beginning weight (-0.9 to 0.9) and set learning rate
net.set_random_constraint(.9)
net.set_learnrate(.1)

net.set_all_inputs(trainX)
net.set_all_targets(trainY)

length = len(trainX)
learn_end_point = int(length * .8)

net.set_learn_range(0, learn_end_point)
net.set_test_range(learn_end_point + 1, length - 1)

net.init_layers(input_nodes, [hidden_nodes], output_nodes,
    NARXRecurrent(
        output_order,
        incoming_weight_from_output,
        input_order,
        incoming_weight_from_input))

net.randomize_network()

In [ ]:
# set the activation function
net.layers[1].set_activation_type('tanh')

# set number of epochs, start learning
num_epochs = 200
net.learn(epochs=num_epochs, show_epoch_results=True,
    random_testing=False)

# print loss
mse = net.test()
print "test mse = ", mse